## Install KPI Reporter

For this tutorial, we will only need a few plugins. You can install `kpireport[all]` to install all available plugins.

In [1]:
pip install kpireport kpireport-mysql kpireport-plot

## Set up test database

For this tutorial, we'll be using a test database on a local MySQL server. The [initdb.sql](./initdb.sql) file has the test data; you can update this file and re-run this cell to re-seed the local database.

In [10]:
# Start the server in the background
[[ -f /tmp/mysql.pid ]] || (mysqld &) && while [[ ! -f /tmp/mysql.pid ]]; do sleep 1; done
# Initialize the database and tables
mysql <initdb.sql && echo "Database initialized."

2021-01-24T01:24:00.601626Z mysqld_safe Logging to '/tmp/mysql_error.log'.
2021-01-24T01:24:00.630732Z mysqld_safe Starting mysqld daemon with databases from /tmp/mysql
Database initialized.


## Run your first report

The [`config.yaml`](./config.yaml) file contains our report configuration. We will start by defining a **datasource** for our MariaDB server and a **view** displaying the result of a query as a bar chart.

In [11]:
cat config.yaml

---
title: Quickstart Tutorial

datasources:
    my_db:
        plugin: mysql
        args:
            host: localhost
            user: ${NB_USER}
            
views:
    num_users:
        plugin: plot
        args:
            datasource: my_db
            query: select * from tutorial.new_users
            kind: bar
            
outputs:
    html:
        plugin: static


By default, the report will be rendered as HTML using the [Static](https://kpi-reporter.readthedocs.io/en/latest/plugins/static.html) plugin.

In [13]:
kpireport -c config.yaml

INFO:kpireport.plugin:Loaded datasource plugins: ['mysql', 'jenkins', 'prometheus']
INFO:kpireport.plugin:Initialized datasource my_db
INFO:kpireport.plugin:Loaded view plugins: ['jenkins.build_summary', 'plot', 'single_stat', 'prometheus.alert_summary']
INFO:kpireport.plugin:Initialized view num_users
INFO:kpireport.plugin:Loaded output driver plugins: ['static', 'slack', 's3', 'scp', 'sendgrid', 'smtp']
INFO:kpireport.plugin:Initialized output driver html
INFO:kpireport.report:Sending report via output driver html
Generated report in 1439.04ms.


## Examine output

**[View output HTML](./_build/latest-quickstart-tutorial/index.html)**

## Adding a new view

We will now add a new view that shows the total number of users in the database, using the [Single stat](https://kpi-reporter.readthedocs.io/en/latest/plugins/plot.html#single-stat) plugin.


In [15]:
cat config-2.yaml

---
title: Quickstart Tutorial

datasources:
    my_db:
        plugin: mysql
        args:
            host: localhost
            user: ${NB_USER}
            
views:
    num_users:
        cols: 4
        plugin: plot
        args:
            datasource: my_db
            query: select * from tutorial.new_users
            kind: bar
    total_users:
        title: Total new users
        cols: 2
        plugin: single_stat
        args:
            datasource: my_db
            query: select sum(num_new_users) from tutorial.new_users
            
outputs:
    html:
        plugin: static


In [16]:
kpireport -c config-2.yaml

INFO:kpireport.plugin:Loaded datasource plugins: ['mysql', 'jenkins', 'prometheus']
INFO:kpireport.plugin:Initialized datasource my_db
INFO:kpireport.plugin:Loaded view plugins: ['jenkins.build_summary', 'plot', 'single_stat', 'prometheus.alert_summary']
INFO:kpireport.plugin:Initialized view num_users
INFO:kpireport.plugin:Initialized view total_users
INFO:kpireport.plugin:Loaded output driver plugins: ['static', 'slack', 's3', 'scp', 'sendgrid', 'smtp']
INFO:kpireport.plugin:Initialized output driver html
INFO:kpireport.report:Sending report via output driver html
Generated report in 1346.79ms.


**[View output HTML](./_build/latest-quickstart-tutorial/index.html)**

Notice that we added a `title` to the single stat view to give more context as to what the number indicates. Additionally, the view was placed side-by-side with the first plot by adjusting the `columns` parameter for each view. By default every report uses a 6-column layout, but this [can be configured at the theme level](https://kpi-reporter.readthedocs.io/en/latest/api/report.html#kpireport.report.Theme.num_columns).

## Have fun!

This example has hopefully given you some ideas on how to get started creating your first report :)
If you are interested in not outputting HTML, but instead sending via email, refer to the plugin documentation for, e.g., the [SMTP](https://kpi-reporter.readthedocs.io/en/latest/plugins/smtp.html) or [SendGrid](https://kpi-reporter.readthedocs.io/en/stable/plugins/sendgrid.html) plugins.